In [2]:
import os
from bs4 import BeautifulSoup as bs
#import requests
#from splinter import Browser
#from splinter.exceptions import ElementDoesNotExist
import pandas as pd
import pymongo

In [3]:
url = 'https://en.wikipedia.org/wiki/Arizona'
page_df = pd.read_html(url, header=0)
elec_df= page_df[16]

In [4]:
elec_df.head()

,Year,Republican,Democratic
0,2016,"49.15% 1,240,656","45.35% 1,144,709"
1,2012,"53.65% 1,233,654","44.59% 1,025,232"
2,2008,"53.60% 1,230,111","45.12% 1,034,707"
3,2004,"54.87% 1,104,294","44.40% 893,524"
4,2000,"50.95% 781,652","44.67% 685,341"


In [6]:
elec_df[["Rep %", "junk"]] = elec_df["Republican"].str.split("%",expand=True) 
elec_df[["Dem %","junk2"]] = elec_df["Democratic"].str.split("%",expand=True) 
elec_df.head()

,Year,Republican,Democratic,Rep %,junk,Dem %,junk2
0,2016,"49.15% 1,240,656","45.35% 1,144,709",49.15,"1,240,656",45.35,"1,144,709"
1,2012,"53.65% 1,233,654","44.59% 1,025,232",53.65,"1,233,654",44.59,"1,025,232"
2,2008,"53.60% 1,230,111","45.12% 1,034,707",53.60,"1,230,111",45.12,"1,034,707"
3,2004,"54.87% 1,104,294","44.40% 893,524",54.87,"1,104,294",44.40,"893,524"
4,2000,"50.95% 781,652","44.67% 685,341",50.95,"781,652",44.67,"685,341"


In [7]:
cleandf = elec_df[["Year","Rep %", "Dem %"]].set_index("Year")
cleandf = cleandf.rename(columns={"Rep %":"Replublican %", "Dem %":"Democratic %"})
cleandf.head()

,Replublican %,Democratic %
Year,,
2016,49.15,45.35
2012,53.65,44.59
2008,53.60,45.12
2004,54.87,44.40
2000,50.95,44.67


In [8]:
demoData = pd.read_csv("PopulationRaceData.csv")
demoData["Population"] = demoData["Population"].astype(int)
demoData["White"] = demoData["White"].astype(int)
demoData["Black"] = demoData["Black"].astype(int)
demoData["Hispanic"] = demoData["Hispanic"].astype(int)
demoData["American Indian"] = demoData["American Indian"].astype(int)
demoData["Asian"] = demoData["Asian"].astype(int)
demoData["Other"] = demoData["Other"].astype(int)
demoData.head()

,Year,Population,White,Black,Hispanic,American Indian,Asian,Other
0,1980,2718215,2026262,73245,440701,145106,2026,30855
1,1981,2777403,2063757,75217,456178,147917,5120,29193
2,1982,2895779,2138747,79163,487132,153540,11308,25871
3,1983,3014156,2213737,83108,518087,159163,17496,22548
4,1984,3132533,2288728,87054,549042,164786,23684,19226


In [9]:
join_df = cleandf.merge(demoData,how='inner', on='Year')
join_df.head()

,Year,Replublican %,Democratic %,Population,White,Black,Hispanic,American Indian,Asian,Other
0,2008,53.60,45.12,6155507,3616636,222383,1782736,252915,155285,125549
1,2004,54.87,44.40,5682487,3458615,188948,1557912,243894,124837,108279
2,2000,50.95,44.67,5130632,3274258,149941,1295617,233370,89315,88131
3,1996,44.29,46.52,4489517,2990726,130195,1029932,214435,72784,51444
4,1992,38.47,36.52,3939991,2747698,113271,802202,198205,58614,19998


In [10]:
join_df.set_index("Year", inplace=True)
join_df.head()

,Replublican %,Democratic %,Population,White,Black,Hispanic,American Indian,Asian,Other
Year,,,,,,,,,
2008,53.60,45.12,6155507,3616636,222383,1782736,252915,155285,125549
2004,54.87,44.40,5682487,3458615,188948,1557912,243894,124837,108279
2000,50.95,44.67,5130632,3274258,149941,1295617,233370,89315,88131
1996,44.29,46.52,4489517,2990726,130195,1029932,214435,72784,51444
1992,38.47,36.52,3939991,2747698,113271,802202,198205,58614,19998


In [11]:
join_html = join_df.to_html()
join_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Replublican %</th>\n      <th>Democratic %</th>\n      <th>Population</th>\n      <th>White</th>\n      <th>Black</th>\n      <th>Hispanic</th>\n      <th>American Indian</th>\n      <th>Asian</th>\n      <th>Other</th>\n    </tr>\n    <tr>\n      <th>Year</th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>2008</th>\n      <td>53.60</td>\n      <td>45.12</td>\n      <td>6155507</td>\n      <td>3616636</td>\n      <td>222383</td>\n      <td>1782736</td>\n      <td>252915</td>\n      <td>155285</td>\n      <td>125549</td>\n    </tr>\n    <tr>\n      <th>2004</th>\n      <td>54.87</td>\n      <td>44.40</td>\n      <td>5682487</td>\n      <td>3458615</td>\n      <td>188948</td>\n      <td>1557912</td>\n      <td>243894

In [13]:
join_dict = {"cdata" : join_html}
join_dict

{'cdata': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Replublican %</th>\n      <th>Democratic %</th>\n      <th>Population</th>\n      <th>White</th>\n      <th>Black</th>\n      <th>Hispanic</th>\n      <th>American Indian</th>\n      <th>Asian</th>\n      <th>Other</th>\n    </tr>\n    <tr>\n      <th>Year</th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>2008</th>\n      <td>53.60</td>\n      <td>45.12</td>\n      <td>6155507</td>\n      <td>3616636</td>\n      <td>222383</td>\n      <td>1782736</td>\n      <td>252915</td>\n      <td>155285</td>\n      <td>125549</td>\n    </tr>\n    <tr>\n      <th>2004</th>\n      <td>54.87</td>\n      <td>44.40</td>\n      <td>5682487</td>\n      <td>3458615</td>\n      <td>188948</td>\n      <td>1557912</td>\n      

In [14]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.demoelection
collection = db.combData

In [98]:
#cdata = join_df.to_dict(orient='records')

In [16]:
collection.drop()
collection.insert_one(join_dict)


In [ ]:
#junk catcher cell. DO NOT EXECUTE

#url = 'https://uselectionatlas.org/RESULTS/'
#browser.visit(url)

# soup = bs(response.text, 'lxml')
# table = soup.find_all('table', class_='wikitable')

#response = requests.get(url)

elec_df = pd.read_html(elecTable)
#elec_df
#print(elecTable)

elecTable = table[9].text
#print(elecTable)

!which chromedriver
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.demoelection
collection = db.election
data = cleandf.to_dict(orient='records')
collection.insert_many(data)
for rec in collection.find():
    print(rec)

join_df.to_csv(path_or_buf='./joined.csv', index=True)